In [1]:
import sys
import warnings
import numpy as np
import scipy as sc
import scipy.interpolate as sci
from datetime import date
import itertools
flatten = itertools.chain.from_iterable
#import own functions from utils.py
from utils import *

In [2]:
# setup testing coefficients
coefB = 0.2; #coef determining beta (transmission rate)
coefG = 0.6; #coef determining gamma  (removal rate)
coefBrx = 0.2; #coef determining beta (transmission rate) relaxed after 3rd date
coefGrx = 0.2; #coef determining gamma (removal rate) relaxed after 3rd date

betaNode = [0.8,0.8,0.4,0.1];
gammaNode = [0.1,0.1,0.3,0.9];

# derive other coefficients
grid1 = [0.2, 1, 1.4];
grid2 = [1.4, 1, 0.2];
coefBamp = sci.CubicSpline(grid1, grid2, axis=0, bc_type='not-a-knot')(coefB).tolist() # Matlab's csapi uses the not-a-knot end condition

ctrBeta0 = 0.8;
ctrBeta =  coefBamp*betaNode[-2];
ctrBeta = [ctrBeta,(ctrBeta0+coefBrx)*ctrBeta];

ctrGamma0 = 1.2;
ctrGamma = coefG*gammaNode[-1];
ctrGamma = [ctrGamma,(ctrGamma0-coefGrx)*ctrGamma] ; 



In [3]:
# setup testing coefficients for SEIR prediction function

RK = 1;
N = 1000000; 
# simulate dates from 8/03/2020 to 13/04/2020
Dates = range(date.toordinal(date(2020,3,1)), date.toordinal(date(2020,4,13))+1);
StartDate = Dates[0];
DaysHistoryFromStart = 55;
EndDate0 = Dates[-1]; # take this to be the last available date of the given data
EndDate1 = EndDate0+DaysHistoryFromStart-1;
t_step = 1;
Nodes1 = [Dates[5], Dates[10]];
Nodes = list(flatten([[StartDate], Nodes1, [EndDate0]]) ); # this assumption may have to be changed
Sigma = 1/11.5;
E0 = 0; 
I0 = 5; #newly infected
R0 = 2;
S0 = N-I0-E0-R0; 
SPLINE = 'pchip' ; 

SetDates0 = [5,15,25]
SetDates1 = [x+EndDate0 for x in SetDates0]
Tbreak=SetDates1[0]



In [4]:
[S,E,I,R,gamma] = seir_spline_predict(StartDate, EndDate0, EndDate1, Tbreak, ctrBeta, ctrGamma, Nodes, betaNode, gammaNode, N, S0, E0, I0, R0, SPLINE, Sigma, RK)


[999993,
 999989.1504849202,
 999985.3936468563,
 999981.501364058,
 999977.2751554121,
 999972.5300812818,
 999967.6909420067,
 999963.4332868643,
 999959.7236364405,
 999956.3613228926,
 999953.1427843429,
 999950.4630531986,
 999948.7517963324,
 999947.6882219864,
 999946.9987926965,
 999946.517362806,
 999946.1538167616,
 999945.8605934436,
 999945.6122872486,
 999945.3949097933,
 999945.2004452129,
 999945.0240768397,
 999944.8627464145,
 999944.7143882548,
 999944.5775134007,
 999944.4509795012,
 999944.3338612849,
 999944.2253764059,
 999944.1248421874,
 999944.0316497954,
 999943.945248343,
 999943.8651347093,
 999943.790846684,
 999943.7219580814,
 999943.6580750415,
 999943.5988330741,
 999943.5438945844,
 999943.4929467295,
 999943.4456995153,
 999943.4018840785,
 999943.3612511207,
 999943.3235694697,
 999943.288624754,
 999943.2562181773,
 999943.1979380243,
 999943.0873459601,
 999942.9321462321,
 999942.7442337095,
 999942.5363959589,
 999942.3182963108,
 999942.09439510

In [7]:
print(gamma)

[0.1        0.1        0.1        0.1        0.1        0.1
 0.17692535 0.23582801 0.27282442 0.29224249 0.3        0.51624992
 0.65509043 0.74405552 0.80094364 0.83724079 0.86034642 0.87501866
 0.88431134 0.89018049 0.89387634 0.8961962  0.89764735 0.8985517
 0.89911301 0.89945986 0.89967315 0.89980361 0.89988294 0.89993085
 0.89995957 0.89997665 0.8999867  0.89999255 0.89999591 0.89999781
 0.89999886 0.89999943 0.89999973 0.89999988 0.89999996 0.89999999
 0.9        0.9        0.75621843 0.64481946 0.57816733 0.54755901
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       0.54       0.54
 0.54       0.54       0.54       0.54       